In [1]:
#-*- coding: utf-8 -*-

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from pandas import DataFrame, Series

%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
phone_brand_device_model = pd.read_csv('data/phone_brand_device_model.csv')
gender_age_train = pd.read_csv('data/gender_age_train.csv')
label_categories = pd.read_csv('data/label_categories.csv')
events = pd.read_csv('data/events.csv')
app_labels = pd.read_csv('data/app_labels.csv')
app_events = pd.read_csv('data/app_events.csv', dtype = {'event_id' : np.int8, 'app_id' : np.int64, 
                                                   'is_installed' : np.int8, 'is_active' : np.int8})
gender_age_test = pd.read_csv('data/gender_age_test.csv')

## 브랜드명 전환

In [3]:
english_phone_brands_mapping = {
    "三星": "samsung",
    "天语": "Ktouch",
    "海信": "hisense",
    "联想": "lenovo",
    "欧比": "obi",
    "爱派尔": "ipair",
    "努比亚": "nubia",
    "优米": "youmi",
    "朵唯": "dowe",
    "黑米": "heymi",
    "锤子": "hammer",
    "酷比魔方": "koobee",
    "美图": "meitu",
    "尼比鲁": "nibilu",
    "一加": "oneplus",
    "优购": "yougo",
    "诺基亚": "nokia",
    "糖葫芦": "candy",
    "中国移动": "ccmc",
    "语信": "yuxin",
    "基伍": "kiwu",
    "青橙": "greeno",
    "华硕": "asus",
    "夏新": "panosonic",
    "维图": "weitu",
    "艾优尼": "aiyouni",
    "摩托罗拉": "moto",
    "乡米": "xiangmi",
    "米奇": "micky",
    "大可乐": "bigcola",
    "沃普丰": "wpf",
    "神舟": "hasse",
    "摩乐": "mole",
    "飞秒": "fs",
    "米歌": "mige",
    "富可视": "fks",
    "德赛": "desci",
    "梦米": "mengmi",
    "乐视": "lshi",
    "小杨树": "smallt",
    "纽曼": "newman",
    "邦华": "banghua",
    "E派": "epai",
    "易派": "epai",
    "普耐尔": "pner",
    "欧新": "ouxin",
    "西米": "ximi",
    "海尔": "haier",
    "波导": "bodao",
    "糯米": "nuomi",
    "唯米": "weimi",
    "酷珀": "kupo",
    "谷歌": "google",
    "昂达": "ada",
    "聆韵": "lingyun",
    "小米": "Xiaomi",
    "华为": "Huawei",
    "魅族": "Meizu",
    "中兴": "ZTE",
    "酷派": "Coolpad",
    "金立": "Gionee",
    "SUGAR": "SUGAR",
    "OPPO": "OPPO",
    "vivo": "vivo",
    "HTC": "HTC",
    "LG": "LG",
    "ZUK": "ZUK",
    "TCL": "TCL",
    "LOGO": "LOGO",
    "SUGAR": "SUGAR",
    "Lovme": "Lovme",
    "PPTV": "PPTV",
    "ZOYE": "ZOYE",
    "MIL": "MIL",
    "索尼" : "Sony",
    "欧博信" : "Opssom",
    "奇酷" : "Qiku",
    "酷比" : "CUBE",
    "康佳" : "Konka",
    "亿通" : "Yitong",
    "金星数码" : "JXD",
    "至尊宝" : "Monkey King",
    "百立丰" : "Hundred Li Feng",
    "贝尔丰" : "Bifer",
    "百加" : "Bacardi",
    "诺亚信" : "Noain",
    "广信" : "Kingsun",
    "世纪天元" : "Ctyon",
    "青葱" : "Cong",
    "果米" : "Taobao",
    "斐讯" : "Phicomm",
    "长虹" : "Changhong",
    "欧奇" : "Oukimobile",
    "先锋" : "XFPLAY",
    "台电" : "Teclast",
    "大Q" : "Daq",
    "蓝魔" : "Ramos",
    "奥克斯" : "AUX"
}

phone_brand_device_model.phone_brand = phone_brand_device_model.phone_brand.map(pd.Series(english_phone_brands_mapping), na_action='ignore')

### time saperate year / month / day / hour

In [4]:
events['timestamp'].replace('-', '', regex = True, inplace = True)
events['timestamp'].replace(':', '', regex = True, inplace = True)
events['timestamp'].replace(' ', '', regex = True, inplace = True)

In [5]:
events['year'] = events['timestamp'].map(lambda x: x[0:4])
events['month'] = events['timestamp'].map(lambda x: x[4:6])
events['day'] = events['timestamp'].map(lambda x: x[6:8])
events['time'] = events['timestamp'].map(lambda x: x[8:10])

In [6]:
events.drop(['timestamp'], axis = 1, inplace = True)
events.drop(['year'], axis = 1, inplace = True)

### longitude / latitude mean값으로 변경
* dvice_id에 따른 지역 편차 줄임

In [8]:
event_mean = events.groupby('device_id', as_index = False).agg({'longitude' : 'mean', 
                                                   'latitude' : 'mean'})

In [36]:
event_mean.head()

,device_id,longitude,latitude
0,-9222956879900151005,113.240000,23.190000
1,-9222661944218806987,NaN,NaN
2,-9222399302879214035,NaN,NaN
3,-9221825537663503111,113.446735,34.204592
4,-9221767098072603291,NaN,NaN


In [9]:
events_tude_mean = pd.merge(events, event_mean, on = 'device_id')

In [10]:
events_tude_mean = events_tude_mean.drop('longitude_x', 1)
events_tude_mean = events_tude_mean.drop('latitude_x', 1)

In [11]:
events_tude_mean.sort_values(by = 'device_id').head(100)

,event_id,device_id,month,day,time,longitude_y,latitude_y
1510824,661623,-9222956879900151005,05,07,11,90.592000,18.552000
1510850,2085017,-9222956879900151005,05,06,15,90.592000,18.552000
1510849,2085015,-9222956879900151005,05,06,15,90.592000,18.552000
1510848,2084996,-9222956879900151005,05,06,15,90.592000,18.552000
1510847,2068832,-9222956879900151005,05,07,12,90.592000,18.552000
1510846,1794538,-9222956879900151005,05,07,07,90.592000,18.552000
1510845,1772031,-9222956879900151005,05,07,11,90.592000,18.552000
1510851,2111353,-9222956879900151005,05,07,12,90.592000,18.552000
1510844,1747268,-9222956879900151005,05,07,12,90.592000,18.552000
1510842,1699936,-9222956879900151005,05,06,21,90.592000,18.552000


### 명목형 feature - one hot encoding (brand name)

In [12]:
from sklearn.preprocessing import OneHotEncoder

In [20]:
onehot_brand = pd.get_dummies(phone_brand_device_model, sparse=False)

In [21]:
onehot_brand.head()

device_id  phone_brand_AUX  phone_brand_Bacardi  \
0 -8890648629457979026                0                    0   
1  1277779817574759137                0                    0   
2  5137427614288105724                0                    0   
3  3669464369358936369                0                    0   
4 -5019277647504317457                0                    0   

   phone_brand_Bifer  phone_brand_CUBE  phone_brand_Changhong  \
0                  0                 0                      0   
1                  0                 0                      0   
2                  0                 0                      0   
3                  0                 0                      0   
4                  0                 0                      0   

   phone_brand_Cong  phone_brand_Coolpad  phone_brand_Ctyon  phone_brand_Daq  \
0                 0                    0                  0                0   
1                 0                    0                  0                0   
2                 0                    0                  0                0   
3                 0                    0                  0                0   
4                 0                    0                  0                0   

   phone_brand_Gionee  phone_brand_HTC  phone_brand_Huawei  \
0                   0                0                   0   
1                   0                0                   0   
2                   0                0                   0   
3                   0                0                   0   
4                   0                0                   0   

   phone_brand_Hundred Li Feng  phone_brand_JXD  phone_brand_Kingsun  \
0                            0                0                    0   
1                            0                0                    0   
2                            0                0                    0   
3                            0                0                    0   
4                            0                0                    0   

   phone_brand_Konka  phone_brand_Ktouch  phone_brand_LG  phone_brand_LOGO  \
0                  0                   0               0                 0   
1                  0                   0               0                 0   
2                  0                   0               0                 0   
3                  0                   0               0                 0   
4                  0                   0               0                 0   

   phone_brand_Lovme  phone_brand_MIL  phone_brand_Meizu  \
0                  0                0                  0   
1                  0                0                  0   
2                  0                0                  0   
3                  0                0                  0   
4                  0                0                  0   

   phone_brand_Monkey King  phone_brand_Noain  phone_brand_OPPO  \
0                        0                  0                 0   
1                        0                  0                 0   
2                        0                  0                 0   
3                        0                  0                 0   
4                        0                  0                 0   

   phone_brand_Opssom  phone_brand_Oukimobile  phone_brand_PPTV  \
0                   0                       0                 0   
1                   0                       0                 0   
2                   0                       0                 0   
3                   0                       0                 0   
4                   0                       0                 0   

   phone_brand_Phicomm  phone_brand_Qiku  phone_brand_Ramos  \
0                    0                 0                  0   
1                    0                 0                  0   
2                    0                 0                  0   
3                    0                 0                  0   

In [22]:
onehot_brand.shape

(187245, 1696)

### 명목형 feature  - label encoding (brand name)

In [23]:
from sklearn.preprocessing import LabelEncoder

In [24]:
le = LabelEncoder()

In [43]:
phone_brand_device_model.phone_brand.dropna().max()

'yuxin'

In [41]:
# 누락된 phone_brand data가 있었다!!!!!!

phone_brand_device_model.phone_brand.fillna(phone_brand_device_model.phone_brand.dropna().max(), inplace = True)


In [42]:
phone_brand_device_model['brand'] = le.fit_transform(phone_brand_device_model['phone_brand'])

In [46]:
phone_brand_device_model.device_model.max()

'黄金斗士青春版'

In [48]:
phone_brand_device_model.device_model.fillna(phone_brand_device_model.device_model.dropna().max(), inplace = True)

In [51]:
phone_brand_device_model['device'] = le.fit_transform(phone_brand_device_model['device_model'])

In [52]:
phone_brand_device_model.head()

,device_id,phone_brand,device_model,brand,device
0,-8890648629457979026,Xiaomi,红米,37,1517
1,1277779817574759137,Xiaomi,MI 2,37,749
2,5137427614288105724,samsung,Galaxy S4,85,560
3,3669464369358936369,SUGAR,时尚手机,31,1503
4,-5019277647504317457,samsung,Galaxy Note 2,85,536


In [53]:
phone_brand_label = phone_brand_device_model.copy()

In [54]:
phone_brand_label = phone_brand_label.drop('phone_brand', 1)
phone_brand_label = phone_brand_label.drop('device_model', 1)

## train set과 app 정보 결합  (일단 nan 치환하지 말것)

In [4]:
# longitude, latitude 0값을 nan으로 치환
events['longitude'] = events['longitude'].map(lambda x : np.nan if x == 0.00 else x)
events['latitude'] = events['latitude'].map(lambda x : np.nan if x == 0.00 else x)

## app_event 분할

In [5]:
# app_events size가 너무 커서 분할(6등분)

app_events1 = app_events.iloc[:5412195,:]

In [7]:
app_events1.head()

,event_id,app_id,is_installed,is_active
0,2,5927333115845830913,1,1
1,2,-5720078949152207372,1,0
2,2,-1633887856876571208,1,0
3,2,-653184325010919369,1,1
4,2,8693964245073640147,1,1


## age - app 결합

### gender_age_train & events & app_events1  MERGE
* step1 app 별 category연결

In [12]:
g_app_label = pd.merge(app_events1, app_labels, on = 'app_id')

In [13]:
g_app_cate = pd.merge(g_app_label, label_categories, on = 'label_id')

In [14]:
g_app_cate = g_app_cate.sort_values(by = 'event_id')

In [9]:
g_app_cate.head()

,event_id,app_id,is_installed,is_active,label_id,category
29060920,-128,6666573791286858743,1,1,959,financial
28764859,-128,4946688067929487794,1,0,794,Tencent
28764896,-128,4946688067929487794,1,0,794,Tencent
325539,-128,8693964245073640147,1,1,549,Property Industry 1.0
6323410,-128,4348659952760821294,1,0,548,Industry tag


###  active event만 사용  &  중복 label 제거

In [10]:
g_app_cate = g_app_cate[np.asarray(g_app_cate.is_active != 0)]

In [11]:
g_app_cate_uniq = g_app_cate.drop_duplicates(['event_id', 'label_id'], keep = 'first')

In [15]:
# event_id minus 값은 어디에 해당하는 것인가??? train, test에 해당되지 않음

g_app_cate_uniq.head(20)

,event_id,app_id,is_installed,is_active,label_id,category
29060920,-128,6666573791286858743,1,1,959,financial
325539,-128,8693964245073640147,1,1,549,Property Industry 1.0
6323248,-128,4348659952760821294,1,1,548,Industry tag
11044997,-128,-329259848532828564,1,1,302,unknown
24193743,-128,6965654211116534216,1,1,761,Financial Services
34571543,-128,1518384689099456437,1,1,167,love and marriage
16743055,-128,6666573790395438502,1,1,756,Internet banking
18664260,-128,-795631016982407721,1,1,779,Higher income
28766865,-128,499698078258542214,1,1,794,Tencent
17372394,-128,-3690515741167825076,1,1,757,P2P


g_app_cate_s = g_app_cate.groupby('event_id')['is_active'].agg('sum').reset_index()

g_app_cate.shape

g_app_cate_s.tail(20)

events.sort_values(by = 'device_id').head(100)

app_events.sort_values(by = 'event_id').head(100)

### 위치정보와 category 연결

In [16]:
time_cate = pd.merge(events, g_app_cate_uniq, on='event_id')

In [17]:
time_cate.head(200)

,event_id,device_id,timestamp,longitude,latitude,app_id,is_installed,is_active,label_id,category
0,1,29182687948017175,2016-05-01 00:55:25,121.38,31.24,8483751493632839871,1,1,237,store management
1,1,29182687948017175,2016-05-01 00:55:25,121.38,31.24,7167114343576723123,1,1,730,And the Church
2,1,29182687948017175,2016-05-01 00:55:25,121.38,31.24,5195654633279707117,1,1,548,Industry tag
3,1,29182687948017175,2016-05-01 00:55:25,121.38,31.24,-5103071956854770842,1,1,854,Property Industry new
4,1,29182687948017175,2016-05-01 00:55:25,121.38,31.24,4742656071525783405,1,1,774,High mobility
5,1,29182687948017175,2016-05-01 00:55:25,121.38,31.24,5070413844955847732,1,1,775,Liquid medium
6,1,29182687948017175,2016-05-01 00:55:25,121.38,31.24,506977247017159424,1,1,757,P2P
7,1,29182687948017175,2016-05-01 00:55:25,121.38,31.24,-2602537546870821175,1,1,131,"online shopping, price comparing"
8,1,29182687948017175,2016-05-01 00:55:25,121.38,31.24,6666573783032938001,1,1,948,Booking channels
9,1,29182687948017175,2016-05-01 00:55:25,121.38,31.24,-4866707337108514028,1,1,706,Customized 1
